In [1]:
import numpy as np 
import pandas as pd 
import requests 
import lxml.html as lh
print("Importing is done")

Importing is done


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_content = lh.fromstring(requests.get(url).content)
table_data = page_content.xpath("//tr")

In [3]:
toronto_data=[]
col_ind=0
for th in table_data[0]:
    col_ind+=1
    header=str(th.text_content()).replace('\n','')
    print (col_ind,header)
    toronto_data.append((header,[]))
    #print(toronto_data)

1 Postal Code
2 Borough
3 Neighbourhood


In [4]:
for td in range(1,len(table_data)):
    row=table_data[td]
    if len(row) != 3:
      break
    
    col_ind=0
    
    for val in row.iterchildren():
        data=str(val.text_content()).replace('\n','')
        #Append the data after header
        toronto_data[col_ind][1].append(data)
        #Increment i for next column
        col_ind+=1

In [5]:
toronto_struct={header:rows for (header,rows) in toronto_data}
toronto_df=pd.DataFrame(toronto_struct)
toronto_df = toronto_df.drop(toronto_df.index[toronto_df['Borough'] == 'Not assigned']).reset_index(drop = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
toronto_df.shape

(104, 3)

In [7]:
toronto_df = toronto_df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_df.loc[toronto_df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_df['Borough']
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [8]:
toronto_df.to_csv(r'toronto_data.csv', index=False)